In [3]:
import sys
sys.path.append('/home/tongwade780/pdb_website/python_code/')
from charge_sasa_dssp_new import *
import os
import csv
import numpy as np
import pandas as pd
import h5py 
import time
start = time.time()
pdb_path = '/mnt/Tsunami_HHD/Tang/onebyone_clsuter/muscle_alignment_pdb/'
pdbname_file = open ('/home/tongwade780/pdb_website/all_ca_number_all_code/onechain_100clust.csv').readlines ()
next_file = []

for line in pdbname_file[0:]:
    next_file.append(line.rstrip('\r\n' + '').split(','))
real_cluster_num = 0
for cluster in range(2790,2792):
    real_cluster_num = cluster+1
    delet_file = h5py.File('/mnt/Tsunami_HHD/Tang/onebyone_clsuter/cluster_delet_pdb_index/cluster_{0}_delet_pdb_index.mat'.format(real_cluster_num),'r') 
    delet_file['delet_pdb_index']

    cluster_pdb_name = next_file[cluster]
    delet_index = delet_file['delet_pdb_index'].value.T
    cluster_pdb_name = np.asarray(cluster_pdb_name)
    cluster_pdb_name = np.delete(cluster_pdb_name,delet_index[0]-1,None)
    cluster_pdb_name = filter(None, cluster_pdb_name)
    frames = []
    problem_pdb = []
    for pdb_chain_name in cluster_pdb_name:
        pdbname= pdb_chain_name[0:4].lower()
        pdb_file = '{0}{1}'.format(pdbname,'.pdb')
        pdbPATH = os.path.join(pdb_path,pdb_file)
        
        #sasa table
        sasa_result = getSASA_final(pdbPATH)
        #dssp table
        final_dssp_table = getDSSP_final(pdbPATH)
        try:
            #pka table
            pka_final_result = getcharge_pKa(pdbPATH)
        except:
            problem_pdb.append((pdbname,'getcharge_pKa broke'))
            continue
            #load matlab data with dataframe 

        f = h5py.File('/mnt/Tsunami_HHD/Tang/onebyone_clsuter/feature_matrix/{0}_individual_feature.mat'.format(pdbname),'r') 

        datas = f['individual_feature'].value.T
        matlab_data = pd.DataFrame(datas,index=datas[:,0],columns=['resID','eig_vv_1','eig_vv_2','vector_1','Rank_1','vector_2',
                                                  'Rank_2','GNM_co','co_rank','entropy','rmsf_overall','pca_feature',
                                                  'eig_first','eig_sec','shannon_entropy','shannon_entropy20',
                                                  'eig5_eig1','atomic_1','atomic_3','atomic_5','ssbond_matrix',
                                                  'Polarity','charge','stand_Dcom','side_chain_length',
                                                  'ca_len','hole_ss_bond','contact_per_res','gyradius',
                                                  'phobic_percent','RMSF','RMSF_0f_bfactor','shifted_RMSF',
                                                                  'h_bond_group','intrinsicDisorder'])

        #change resID value type to int
        matlab_data['resID'] = matlab_data['resID'].astype('int')

        sasa_result.columns = ['resID','sasa']

        sasa_result['resID'] = sasa_result['resID'].astype('int')

        pka_final_result.columns = ['resID','pka_charge']
        pka_final_result['resID'] = pka_final_result['resID'].astype('int')
        final_dssp_table.columns = ['resID','dssp_result','loop_percent']
        final_dssp_table['resID'] = final_dssp_table['resID'].replace('',np.nan, regex=True)
        final_dssp_table = final_dssp_table[~final_dssp_table['resID'].isnull()]
#         try:
#             final_dssp_table['resID'] = sasa_result['resID'].values
#         except:
#             problem_pdb.append((pdbname,'get_dssp_merge broke'))
#             continue    
        final_dssp_table['resID'] = final_dssp_table['resID'].astype('int')

        #merge matlab feature and python feature
        merged_data = matlab_data.merge(sasa_result,how = 'left',on = 'resID')

        merged_data = merged_data.merge(pka_final_result,how = 'left',on = 'resID')

        merged_data['pka_charge']=merged_data['pka_charge'].fillna(0)

        merged_data = merged_data.merge(final_dssp_table,how = 'left',on = 'resID')
        #add pdb id
        merged_data['PDB_ID'] = pdbname
        #change feature order
        change_order = ['PDB_ID','resID','atomic_1','atomic_3','atomic_5','ssbond_matrix','Polarity',
                        'eig_vv_1','eig_vv_2','vector_1','Rank_1','vector_2','Rank_2',
                        'GNM_co','co_rank','charge','stand_Dcom','side_chain_length','pka_charge',
                        'dssp_result','sasa','entropy','ca_len','hole_ss_bond','rmsf_overall',
                        'contact_per_res','gyradius','pca_feature','eig_first','eig_sec',
                        'shannon_entropy','shannon_entropy20','eig5_eig1','phobic_percent',
                        'loop_percent','h_bond_group','intrinsicDisorder','RMSF','RMSF_0f_bfactor',
                        'shifted_RMSF']

        merged_data = merged_data[change_order]
        frames.append(merged_data)
        
    final_time = time.time()-start    
    if len(problem_pdb) == 0:
        print 'pdb_no_problem'
    else:    
        problem_pdb_data=pd.DataFrame(problem_pdb)
        problem_pdb_data.columns = ['pdbname','probelm']

        problem_pdb_data.to_csv('/home/tongwade780/pdb_website/python_code/all_cluster_problem_pdb_new/problem_pdb_data_cluster_{0}.csv'.format(real_cluster_num),index= False)
    if len(frames) == 0:
        all_cluster_pdb = filter(None, next_file[cluster])

        all_cluster_pdb = pd.DataFrame(all_cluster_pdb)
        all_cluster_pdb.to_csv('/home/tongwade780/pdb_website/python_code/no_pdb_clsuter_new/cluster_{0}_no_pdb.csv'.format(real_cluster_num),index = False)
            
    else:
        result = pd.concat(frames)
        result.to_csv('/home/tongwade780/pdb_website/python_code/all_cluster_feature_csv_new/cluster_{0}_all_feature.csv'.format(real_cluster_num),index = False)

propka3.1                                                                                    2017-08-18
-------------------------------------------------------------------------------------------------------
--                                                                                                   --
--                                   PROPKA: A PROTEIN PKA PREDICTOR                                 --
--                                                                                                   --
--                                 VERSION 1.0,  04/25/2004, IOWA CITY                               --
--                                             BY HUI LI                                             --
--                                                                                                   --
--                            VERSION 2.0,  11/05/2007, IOWA CITY/COPENHAGEN                         --
--                                BY DELPHINE C. BAS AND DAVID M

In [17]:
clsuter_range = range(2024,2025)+range(2026,2133)+range(2134,2760)+range(2761,2768)+range(2769,2785)+range(2786,2790)

In [19]:
range(2790,2791)


[2790]